This notebook explores methods for comparing two different textual datasets to identify the terms that are distinct to each one:

* Difference of proportions (described in [Monroe et al. 2009, Fighting Words](http://languagelog.ldc.upenn.edu/myl/Monroe.pdf) section 3.2.2
* Mann-Whitney rank-sums test (described in [Kilgarriff 2001, Comparing Corpora](https://www.sketchengine.eu/wp-content/uploads/comparing_corpora_2001.pdf), section 2.3)

In [1]:
import sys, operator
from collections import Counter
from scipy.stats import mannwhitneyu

In [2]:
# the convote data is already tokenized so just split on whitespace
repub_tokens=open("../data/repub.convote.txt", encoding="utf-8").read().split(" ")
dem_tokens=open("../data/dem.convote.txt", encoding="utf-8").read().split(" ")

Q1: First, calculate the simple "difference of proportions" measure from Monroe et al.'s "Fighting Words", section 3.2.2.  What are the top ten terms in this measurement that are most republican and most democrat?

In [36]:
#TODO: remove punctuation
def difference_of_proportions(one_tokens, two_tokens):
    # your code here
    freq_dict1 = Counter(one_tokens)
    freq_dict2 = Counter(two_tokens)
    n1 = len(one_tokens)
    n2 = len(two_tokens)
    print(len(freq_dict1.keys()))
    print(len(freq_dict2.keys()))

    diff_dict = {}
    
    for w in set(freq_dict1.keys()).intersection(set(freq_dict2.keys())):
        p1 = freq_dict1[w]/n1
        p2 = freq_dict2[w]/n2
        diff_prop = p1-p2
        diff_dict[w] = diff_prop                                                
    
    ##if the word is not mentioned by the other corpus at all                                 
    for w in set(freq_dict1.keys()).difference(set(freq_dict2.keys())): 
        p1 = freq_dict1[w]/n1
        diff_prop = p1 - 0
        diff_dict[w] = diff_prop
    
    for w in set(freq_dict2.keys()).difference(set(freq_dict1.keys())):
        p2 = freq_dict2[w]/n2
        diff_prop = 0 - p2
        diff_dict[w] = diff_prop
    return diff_dict
        

In [37]:
diff_dict = difference_of_proportions(dem_tokens, repub_tokens)

15480
13943


In [38]:
most_repub = sorted(diff_dict.items(), key = lambda x: x[1])[:10]
most_dem = sorted(diff_dict.items(), key = lambda x: x[1], reverse = True)[:10]

print("Most Republican words:")
for r in most_repub:
    print(r)
print("****\nMost Democratic words:")    
for d in most_dem:
    print(d)

Most Republican words:
('i', -0.002870948015418236)
('we', -0.0020739540633471117)
('and', -0.0017279456625680124)
('of', -0.0014950519581076668)
(',', -0.00105321588184943)
('chairman', -0.0009598934247981516)
('that', -0.000945583476245123)
('as', -0.0009124972356492223)
('gentleman', -0.0008093810284795912)
('a', -0.0008020309007514565)
****
Most Democratic words:
('not', 0.0015745433184340962)
('$', 0.0015095648428079297)
('cuts', 0.001031315818425968)
('bill', 0.0010228370409021796)
('republican', 0.001001288082839861)
('budget', 0.0009261863664701928)
('billion', 0.0008820967153998979)
('would', 0.0007701123575280444)
('health', 0.0007538336601492987)
('for', 0.0007352277281844066)


Simply analyzing the difference in relative frequencies has a number of downsides: 
1.) As Monroe et al (2009) points out (and we can see here as well), it tends to emphasize high-frequency words (be sure you understand why). 
2.) We're not measuring whether a difference is statistically meaningful or just due to chance; the $\chi^2$ test is one method (described in Kilgarriff 2001 and in the context of collocations in Manning and Schuetze [here](https://nlp.stanford.edu/fsnlp/promo/colloc.pdf)) that addresses the desideratum of finding statistically significant terms, but it too has another downside:
3.) Simply counting up the total number of mentions of a term doesn't account for the "burstiness" of language -- if we see the word "Dracula" in a text, we're probably going to see it again in that same text.  The occurrence of words are not independent random events; they are tightly coupled with each other. If we're trying to understanding the robust differences between two corpora, we might prefer to prioritize words that show up more frequently *everywhere* in corpus A (but not in corpus B) over those that show up only very frequently within narrow slice of A (such as one text in a genre, one chapter in a book, or one speaker when measuring the differences between policital parties).

Q2 (check-plus): One measure that does account for this burstiness is the adaptation by corpus linguistics of the non-parametric Mann-Whitney rank-sum test. The specific adaptation of this test for text is described in Kilgarriff 2001, section 2.3.  Implement this test using a fixed chunk size of 500 and the [scikit-learn mannwhitneyu function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html); what are the top ten terms in this measurement that are most republican and most democrat? 

In [31]:
t = list(mannwhitneyu([1,2,3],[2,3,4]))

In [34]:
t[1]

0.18434413468089078

In [66]:
#TODO: remove punctuation

## Citation: code adapted from https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def divide_chunks(chunk_size, ls):
    for i in range(0, len(ls), chunk_size):
        yield ls[i:i + chunk_size]

def mann_whitney_analysis(one_tokens, two_tokens):
    # your code here
    
    one_chunks = list(divide_chunks(500, one_tokens))
    two_chunks = list(divide_chunks(500, two_tokens))
    
    freq_dict1 = Counter(one_tokens)
    freq_dict2 = Counter(two_tokens)
    
    U_dict ={}

    ## only using words that appear in both corpus
    for word in set(freq_dict1.keys()).intersection(set(freq_dict2.keys())): 
        word_counts1 = []
        word_counts2 = []
        for c1 in one_chunks[:-1]: ## Leftover uneven chunks is discarded
            wc = Counter(c1)
            word_counts1.append(wc[word])
        for c2 in two_chunks[:-1]:
            wc = Counter(c2)
            word_counts2.append(wc[word])
        U_score = list(mannwhitneyu(word_counts1, word_counts2))
        U_dict[word] = U_score
    
    return U_dict



In [85]:
freq_dict1 = Counter(dem_tokens)
freq_dict2 = Counter(repub_tokens)
one_chunks = list(divide_chunks(500, dem_tokens))
two_chunks = list(divide_chunks(500, repub_tokens))
word = next(enumerate(set(freq_dict1.keys()).intersection(set(freq_dict2.keys()))))[1]
word_counts1 = []
word_counts2 = []
U_dict = {}
for c1 in one_chunks[:-1]: ## Leftover uneven chunks is discarded
    wc = Counter(c1)
    word_counts1.append(wc[word])
for c2 in two_chunks[:-1]:
    wc = Counter(c2)
    word_counts2.append(wc[word])
U_score = list(mannwhitneyu(word_counts1, word_counts2))
U_dict[word] = U_score

In [86]:
U_dict

{'': [401544.5, 0.47111929808574066]}

In [67]:
d = mann_whitney_analysis(dem_tokens, repub_tokens)

In [70]:
sorted(d.items(), key = lambda x: x[1])[:115]

[('republican', [301538.0, 1.1826838328124882e-40]),
 ('i', [325948.5, 1.5237584748910261e-12]),
 ('cuts', [327712.5, 3.27973742662632e-31]),
 ('$', [331406.5, 1.9349035907882228e-13]),
 ('not', [338030.5, 1.763811166528947e-09]),
 ('majority', [338080.5, 2.393937075482985e-18]),
 ('billion', [338275.0, 4.1701302576006746e-15]),
 ('republicans', [347467.0, 5.796945335930232e-19]),
 ('gentleman', [348427.5, 1.8181581506276446e-08]),
 ('important', [349640.5, 3.3402805110767186e-09]),
 ('administration', [349664.0, 2.6878084072105137e-13]),
 ('as', [350497.5, 7.934708861032621e-07]),
 ('--', [351024.0, 3.2654438775479844e-14]),
 ('?', [351656.5, 2.980457668426629e-08]),
 ('cut', [351719.5, 5.18595310935567e-16]),
 ('americans', [352241.0, 1.0568244786776509e-09]),
 ('support', [353646.5, 5.202330700143442e-07]),
 ('may', [353850.0, 1.515388968974397e-08]),
 ('chairman', [354063.5, 6.051956905100078e-07]),
 ('opposition', [354284.0, 3.977739558271767e-14]),
 ('thank', [354562.0, 2.8610681

In [71]:
import pickle

In [73]:
pickle.dump(d,open("dict.p", "wb"))